# Feature engineering 1

## Set-up

In [1]:
import os
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector

In [2]:
os.chdir('..')

In [3]:
from src.models.baseline_model import BaselineModel  # Need to import class for creating baseline model (see explanation in 02-Baseline model)
from src.data.utils import COLUMN_DESCRIPTION_DICT

In [4]:
pd.options.display.max_columns = None

## More explicit null imputation for categorical features

In [5]:
train = pd.read_parquet('data/interim/train.parquet')
print(f"Train size: {train.shape}")
dev = pd.read_parquet('data/interim/dev.parquet')
print(f"Dev size: {dev.shape}")

Train size: (184506, 122)
Dev size: (61502, 122)


In [6]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,297783,0,Cash loans,F,N,Y,0,74250.0,112500.0,6282.0,112500.0,Unaccompanied,Working,Incomplete higher,Married,House / apartment,0.018850,-15077,-7915,-2853.0,-8,NaN,1,1,1,1,0,0,Core staff,2.0,2,2,SATURDAY,10,0,0,0,0,0,0,School,NaN,0.598495,0.728141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,1.0,1.0,0.0,-793.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
1,390572,0,Revolving loans,F,Y,Y,2,225000.0,810000.0,40500.0,810000.0,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,0.035792,-14311,-1858,-2306.0,-5196,4.0,1,1,0,1,0,0,High skill tech staff,4.0,2,2,FRIDAY,13,0,0,0,0,0,0,Business Entity Type 3,0.578538,0.491977,0.200926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,0.0,0.0,0.0,0.0,-312.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,212363,0,Cash loans,M,Y,N,0,225000.0,1546020.0,42642.0,1350000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,Municipal apartment,0.032561,-15236,-5751,-7483.0,-4492,8.0,1,1,0,1,1,0,Drivers,2.0,1,1,TUESDAY,16,0,0,0,0,0,0,Business Entity Type 3,0.467365,0.591815,0.762336,0.1103,0.0857,0.9831,0.7688,NaN,0.12,0.1034,0.3333,0.375,NaN,NaN,0.1214,NaN,NaN,0.1124,0.089,0.9831,0.7779,NaN,0.1208,0.1034,0.3333,0.375,NaN,NaN,0.1265,NaN,NaN,0.1114,0.0857,0.9831,0.7719,NaN,0.12,0.1034,0.3333,0.375,NaN,NaN,0.1236,NaN,NaN,reg oper account,block of flats,0.0975,Panel,No,0.0,0.0,0.0,0.0,-1767.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
3,116368,1

In [7]:
object_columns = train.select_dtypes(include=object).columns

train[object_columns].isnull().sum()

NAME_CONTRACT_TYPE                 0
CODE_GENDER                        0
FLAG_OWN_CAR                       0
FLAG_OWN_REALTY                    0
NAME_TYPE_SUITE                  787
NAME_INCOME_TYPE                   0
NAME_EDUCATION_TYPE                0
NAME_FAMILY_STATUS                 0
NAME_HOUSING_TYPE                  0
OCCUPATION_TYPE                57649
WEEKDAY_APPR_PROCESS_START         0
ORGANIZATION_TYPE                  0
FONDKAPREMONT_MODE            126235
HOUSETYPE_MODE                 92655
WALLSMATERIAL_MODE             93890
EMERGENCYSTATE_MODE            87465
dtype: int64

In [13]:
categorical_features_with_nulls = list(train[object_columns].isnull().sum()[train[object_columns].isnull().sum() > 0].index)
categorical_features_with_nulls

['NAME_TYPE_SUITE',
 'OCCUPATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [8]:
application_dat_dict = COLUMN_DESCRIPTION_DICT['application_{train|test}.csv']

In [18]:
for feature in categorical_features_with_nulls:
    print('---------------------------------------------')
    print(feature)
    print('\n')
    print(train[feature].value_counts(dropna=False))
    print('\n')
    print(application_dat_dict[feature])
    print('---------------------------------------------')
    print('\n')

---------------------------------------------
NAME_TYPE_SUITE


Unaccompanied      149065
Family              24087
Spouse, partner      6832
Children             1985
Other_B              1046
NaN                   787
Other_A               543
Group of people       161
Name: NAME_TYPE_SUITE, dtype: int64


{'Description': 'Who was accompanying client when he was applying for the loan', 'Special': nan}
---------------------------------------------


---------------------------------------------
OCCUPATION_TYPE


NaN                      57649
Laborers                 33261
Sales staff              19206
Core staff               16561
Managers                 12871
Drivers                  11190
High skill tech staff     6898
Accountants               5875
Medicine staff            5132
Security staff            4049
Cooking staff             3553
Cleaning staff            2823
Private service staff     1544
Low-skill Laborers        1225
Waiters/barmen staff       792
Secretaries     

1) `NAME_TYPE_SUITE`:

Not clear what the difference is between `Other_A` and `Other_B`. Number of nulls is comparable to these so __map all 3 to `Other`__

2) `OCCUPATION_TYPE`:

Are nulls unemployed?

In [29]:
train['NAME_INCOME_TYPE'].value_counts(dropna=False)

Working                 95016
Commercial associate    43136
Pensioner               33235
State servant           13088
Unemployed                 15
Student                    11
Businessman                 4
Maternity leave             1
Name: NAME_INCOME_TYPE, dtype: int64

In [30]:
train[train['OCCUPATION_TYPE'].isnull()]['NAME_INCOME_TYPE'].value_counts(dropna=False)

Pensioner               33232
Working                 14777
Commercial associate     7369
State servant            2253
Unemployed                 15
Student                     3
Name: NAME_INCOME_TYPE, dtype: int64

Null `OCCUPATION_TYPE` aren't all just unemployed but large group are pensioners.

In [31]:
# Do all unemployed have a null occupation type?
train[train['NAME_INCOME_TYPE'] == 'Unemployed']['OCCUPATION_TYPE'].value_counts(dropna=False)

NaN    15
Name: OCCUPATION_TYPE, dtype: int64

Yes, therefore __if `NAME_INCOME_TYPE` = 'Unemployed' set `OCCUPATION_TYPE` to 'Unemployed'__

In [42]:
train[train['NAME_INCOME_TYPE'] == 'Pensioner']['OCCUPATION_TYPE'].value_counts(dropna=False)

NaN               33232
Sales staff           1
Medicine staff        1
Cleaning staff        1
Name: OCCUPATION_TYPE, dtype: int64

__If `NAME_INCOME_TYPE` = 'Pensioner' and `OCCUPATION_TYPE` is null set `OCCUPATION_TYPE` to 'Pensioner'__

In [32]:
train[train['NAME_INCOME_TYPE'] == 'Commercial associate']['OCCUPATION_TYPE'].value_counts(dropna=False)

Laborers                 7703
NaN                      7369
Sales staff              6669
Managers                 5364
Core staff               3967
Drivers                  2927
Accountants              2403
High skill tech staff    2146
Security staff            952
Cooking staff             765
Medicine staff            623
Private service staff     613
Cleaning staff            558
Waiters/barmen staff      223
Low-skill Laborers        207
Realty agents             206
Secretaries               201
HR staff                  128
IT staff                  112
Name: OCCUPATION_TYPE, dtype: int64

Looks like most Commercial associates have an `OCCUPATION_TYPE` of Laborer, therefore if __if `NAME_INCOME_TYPE` = 'Commercial associate' and `OCCUPATION_TYPE` is null set `OCCUPATION_TYPE` to 'Laborers'__

In [33]:
train[train['NAME_INCOME_TYPE'] == 'State servant']['OCCUPATION_TYPE'].value_counts(dropna=False)

Core staff               4150
NaN                      2253
Medicine staff           1590
Managers                 1328
Laborers                 1136
High skill tech staff     619
Drivers                   503
Accountants               429
Cooking staff             284
Security staff            230
Cleaning staff            158
Secretaries               152
Sales staff               112
HR staff                   40
Waiters/barmen staff       33
IT staff                   32
Private service staff      22
Low-skill Laborers         14
Realty agents               3
Name: OCCUPATION_TYPE, dtype: int64

__If `NAME_INCOME_TYPE` = 'State servant' and `OCCUPATION_TYPE` is null set `OCCUPATION_TYPE` to 'Laborers'__ This is not the most common occupation but laborers seems more generic than core staff

In [43]:
train[train['NAME_INCOME_TYPE'] == 'Working']['OCCUPATION_TYPE'].value_counts(dropna=False)

Laborers                 24419
NaN                      14777
Sales staff              12424
Core staff                8442
Drivers                   7759
Managers                  6176
High skill tech staff     4133
Accountants               3042
Medicine staff            2918
Security staff            2867
Cooking staff             2504
Cleaning staff            2106
Low-skill Laborers        1002
Private service staff      909
Waiters/barmen staff       535
Secretaries                434
Realty agents              246
HR staff                   166
IT staff                   157
Name: OCCUPATION_TYPE, dtype: int64

__If `NAME_INCOME_TYPE` = 'Working' and `OCCUPATION_TYPE` is null set `OCCUPATION_TYPE` to 'Laborers'__

In [44]:
train[train['NAME_INCOME_TYPE'] == 'Student']['OCCUPATION_TYPE'].value_counts(dropna=False)

NaN                     3
Low-skill Laborers      2
Laborers                2
Drivers                 1
Core staff              1
Accountants             1
Waiters/barmen staff    1
Name: OCCUPATION_TYPE, dtype: int64

In [46]:
train[
    (train['OCCUPATION_TYPE'].isnull()) &
    (train['NAME_INCOME_TYPE'] == 'Student')
]['AMT_INCOME_TOTAL']

127522    157500.0
180622    112500.0
182342     81000.0
Name: AMT_INCOME_TOTAL, dtype: float64

Non-zero incomes in these cases so let's __also map to Laborers__

3) `FONDKAPREMONT_MODE`

Let's drop this variable as it is not clear what this is and there is a large proportion of nulls in the data. Interestingly looks like this could point to data being from Kazakhstan (https://www.kaggle.com/c/home-credit-default-risk/discussion/62122)

If monetary values are in the local currency then exchange rate differences explain the large values e.g. 157,500 Kazakhstani Tenge (income of one of the students above) is the equivalent of £300!

## TODO

- Finish mapping of remaining categorical features containing nulls
- In function check if other categorical features have nulls and add assert statement or fill with something
- Look at numeric variables with nulls
- For numeric features with very large proportion of nulls look at definition and see if these are likely to be predictive
- For other features use a standard imputation
- Train LGBM with default hyperparameters on this new preprocessed dataset and see how it performs on Kaggle

- Next notebook (feature engineering 2) bring in other data sources